In [6]:
import logging
import time
from binance.client import Client
from binance.enums import *
from binance.exceptions import BinanceAPIException
import sys

# Setup logging
logging.basicConfig(filename='bot.log', level=logging.INFO,
                    format='%(asctime)s:%(levelname)s:%(message)s')

class BasicBot:
    def __init__(self, api_key, api_secret, testnet=True):
        self.client = Client(api_key, api_secret)

        if testnet:
            self.client.FUTURES_URL = 'https://testnet.binancefuture.com/fapi'
            self.client.API_URL = 'https://testnet.binancefuture.com/fapi'

        # Sync timestamp offset using public methods
        try:
            server_time = self.client.futures_time()['serverTime']
            local_time = int(time.time() * 1000)
            self.client.timestamp_offset = server_time - local_time
            logging.info(f"Time offset set: {self.client.timestamp_offset}ms")
        except Exception as e:
            logging.error(f"Error syncing time: {e}")
            print("Error syncing time:", e)

    def place_order(self, symbol, side, order_type, quantity, price=None):
        try:
            params = {
                'symbol': symbol.upper(),
                'side': SIDE_BUY if side.lower() == 'buy' else SIDE_SELL,
                'type': order_type,
                'quantity': quantity
            }

            if order_type == ORDER_TYPE_LIMIT:
                if price is None:
                    raise ValueError("Price is required for LIMIT orders.")
                params['price'] = price
                params['timeInForce'] = TIME_IN_FORCE_GTC

            logging.info(f"Placing order: {params}")
            response = self.client.futures_create_order(**params)
            logging.info(f"Order response: {response}")
            return response

        except BinanceAPIException as e:
            logging.error(f"Binance API error: {e}")
            return {"error": str(e)}
        except Exception as e:
            logging.error(f"Unexpected error: {e}")
            return {"error": str(e)}

def get_user_input():
    print("=== Binance Futures Testnet Trading Bot ===")
    symbol = input("Enter symbol (e.g., BTCUSDT): ").strip()
    side = input("Buy or Sell? ").strip().lower()
    if side not in ['buy', 'sell']:
        sys.exit("Invalid side. Must be 'buy' or 'sell'.")

    order_type = input("Order type (MARKET / LIMIT): ").strip().upper()
    if order_type not in ['MARKET', 'LIMIT']:
        sys.exit("Invalid order type.")

    try:
        quantity = float(input("Enter quantity: ").strip())
    except ValueError:
        sys.exit("Invalid quantity.")
        
    price = None
    if order_type == 'LIMIT':
        try:
            price = float(input("Enter price: ").strip())
        except ValueError:
            sys.exit("Invalid price.")

    return symbol, side, order_type, quantity, price

if __name__ == "__main__":
    # 📌 Paste your Testnet keys here
    API_KEY = "f8323f72a1c0bdfdd4f19aec5cb395f54401a7978db2d3c0fdb43e77125fe4f3"
    API_SECRET = "231422eb202a3c33a5aa89df89f0c4c97d796abb77d5dffbb359adcd3d0eed17"

    bot = BasicBot(API_KEY, API_SECRET, testnet=True)

    symbol, side, order_type, quantity, price = get_user_input()
    result = bot.place_order(symbol, side, order_type, quantity, price)

    print("Order Result:",result)


=== Binance Futures Testnet Trading Bot ===


Enter symbol (e.g., BTCUSDT):  BTCUSDT
Buy or Sell?  BUY
Order type (MARKET / LIMIT):  LIMIT
Enter quantity:  0.01
Enter price:  62000


Order Result: {'orderId': 4543240579, 'symbol': 'BTCUSDT', 'status': 'NEW', 'clientOrderId': 'x-Cb7ytekJ1268db00a0a62c51e70f1', 'price': '62000.00', 'avgPrice': '0.00', 'origQty': '0.010', 'executedQty': '0.000', 'cumQty': '0.000', 'cumQuote': '0.00000', 'timeInForce': 'GTC', 'type': 'LIMIT', 'reduceOnly': False, 'closePosition': False, 'side': 'BUY', 'positionSide': 'BOTH', 'stopPrice': '0.00', 'workingType': 'CONTRACT_PRICE', 'priceProtect': False, 'origType': 'LIMIT', 'priceMatch': 'NONE', 'selfTradePreventionMode': 'EXPIRE_MAKER', 'goodTillDate': 0, 'updateTime': 1750142386690}
